## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Kaitangata,NZ,-46.2817,169.8464,58.93,77,6,6.22,clear sky
1,1,Nikolskoye,RU,59.7035,30.7861,35.71,93,100,11.05,overcast clouds
2,2,Adrar,MR,20.5022,-10.0711,66.40,10,0,9.40,clear sky
3,3,Andenes,NO,69.3143,16.1194,28.38,74,75,9.22,broken clouds
4,4,Coruripe,BR,-10.1256,-36.1756,79.81,77,1,11.23,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                        (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
4,4,Coruripe,BR,-10.1256,-36.1756,79.81,77,1,11.23,clear sky
12,12,Kapaa,US,22.0752,-159.3190,78.78,73,100,5.99,overcast clouds
19,19,Cayenne,GF,4.9333,-52.3333,80.10,86,100,5.01,overcast clouds
23,23,Pitimbu,BR,-7.4706,-34.8086,80.10,70,4,12.33,clear sky
26,26,Hobart,AU,-42.8794,147.3294,75.02,50,20,1.99,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        164
City           164
Country        164
Lat            164
Lng            164
Max Temp       164
Humidity       164
Cloudiness     164
Wind Speed     164
Description    164
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
4,4,Coruripe,BR,-10.1256,-36.1756,79.81,77,1,11.23,clear sky
12,12,Kapaa,US,22.0752,-159.3190,78.78,73,100,5.99,overcast clouds
19,19,Cayenne,GF,4.9333,-52.3333,80.10,86,100,5.01,overcast clouds
23,23,Pitimbu,BR,-7.4706,-34.8086,80.10,70,4,12.33,clear sky
26,26,Hobart,AU,-42.8794,147.3294,75.02,50,20,1.99,few clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
4,Coruripe,BR,79.81,clear sky,-10.1256,-36.1756,
12,Kapaa,US,78.78,overcast clouds,22.0752,-159.3190,
19,Cayenne,GF,80.10,overcast clouds,4.9333,-52.3333,
23,Pitimbu,BR,80.10,clear sky,-7.4706,-34.8086,
26,Hobart,AU,75.02,few clouds,-42.8794,147.3294,
32,Maumere,ID,82.74,few clouds,-8.6199,122.2111,
43,Saint-Philippe,RE,76.59,overcast clouds,-21.3585,55.7679,
47,San Policarpo,PH,81.72,broken clouds,12.1791,125.5072,
50,Butaritari,KI,82.36,light rain,3.0707,172.7902,
56,Vaini,TO,77.16,light intensity drizzle,-21.2000,-175.2000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found ... skipping")
        

Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping


In [9]:
hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
4,Coruripe,BR,79.81,clear sky,-10.1256,-36.1756,Pousada e Hotel Litoral Sul
12,Kapaa,US,78.78,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
19,Cayenne,GF,80.10,overcast clouds,4.9333,-52.3333,Hôtel Le Dronmi
23,Pitimbu,BR,80.10,clear sky,-7.4706,-34.8086,Reserva do Abiaí
26,Hobart,AU,75.02,few clouds,-42.8794,147.3294,Mantra on Collins Hobart


In [23]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()
clean_hotel_df = hotel_df.dropna()

In [24]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [33]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} & {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [34]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content= hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))